In [ ]:
!pip install tensorflow
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
import re
import keras

from bs4 import BeautifulSoup

import sys
import os

os.environ['KERAS_BACKEND']='tensorflow'

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.utils import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Concatenate, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.layers import Layer, InputSpec
from keras import initializers
from nltk import tokenize
from nltk.corpus import stopwords
from keras.callbacks import EarlyStopping

def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

In [ ]:
#%tensorflow_version 1.15
import tensorflow as tf
tf.__version__

'2.9.2'

In [ ]:
keras.__version__

'2.9.0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Spoiler Net Implementation

In [ ]:
X_train = pd.read_pickle('/content/drive/MyDrive/CIS520 Final Project/X_train.pkl')
X_val = pd.read_pickle('/content/drive/MyDrive/CIS520 Final Project/X_val.pkl')
X_test = pd.read_pickle('/content/drive/MyDrive/CIS520 Final Project/X_test.pkl')

y_train = pd.read_pickle('/content/drive/MyDrive/CIS520 Final Project/y_train.pkl')
y_val = pd.read_pickle('/content/drive/MyDrive/CIS520 Final Project/y_val.pkl')
y_test = pd.read_pickle('/content/drive/MyDrive/CIS520 Final Project/y_test.pkl')

In [ ]:
print(np.sum([y_train == 0]))
print(np.sum([y_train == 1]))

290487
9513


In [ ]:
X_train.shape

(300000, 10)

In [ ]:
data_train = pd.concat([pd.concat([X_train,X_val], axis = 0), pd.concat([y_train, y_val], axis = 0)], axis = 1)
data_train.head(5)

,Unnamed: 0,timestamp,review_id,user_id,book_id,review_rating,sentence,_pos_sentence,_num_sentences,sentence_embedding,is_spoiler
34894,34894,2013-05-08,618798,13045,8859,4,"She has no understanding of why, only that she...",10,21,"[0.12825826, -0.015271957, 0.014717576, 0.0586...",0
65566,65566,2014-04-04,104490,6051,16790,5,Will defiantly be reading more of aly's work,3,4,"[-0.04218291, -0.043608442, 0.017529905, 0.009...",0
126414,126414,2013-09-17,608006,12078,12394,5,"This is not an easy girl meets boy, they giggl...",16,21,"[-0.07032201, 0.008057908, 0.055432968, 0.0241...",0
186864,186864,2016-05-23,1207235,1210,17236,4,-Cesar,28,29,"[-0.06680018, 0.031215038, 0.008926821, 0.0324...",0
1114,1114,2016-09-23,653787,12763,3401,0,love alien romances.,1,3,"[-0.030931823, -0.03957359, 0.047186073, 0.046...",0


In [ ]:
data_train.shape

(350000, 11)

In [ ]:
MAX_SENT_LENGTH = 100
MAX_SENTS = 25
MAX_NB_WORDS = 20000  ## dictionary size
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.15

def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    string = string.strip().lower()
    stopWords = set(stopwords.words('english'))
    trans_string = ' '.join([word for word in string.strip().split() if word not in stopWords])
    return trans_string

In [ ]:
data_train.keys()

Index(['Unnamed: 0', 'timestamp', 'review_id', 'user_id', 'book_id',
       'review_rating', 'sentence', '_pos_sentence', '_num_sentences',
       'sentence_embedding', 'is_spoiler'],
      dtype='object')

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data_train.sentence

34894     She has no understanding of why, only that she...
65566          Will defiantly be reading more of aly's work
126414    This is not an easy girl meets boy, they giggl...
186864                                               -Cesar
1114                                   love alien romances.
                                ...                        
381658    Each new plot line or setting is beautifully c...
312288                                            3.5 Stars
51649     There's an incredible amount of time spent on ...
221593    This was a great addition, and I can't wait to...
160422    This book is much more raw and intense than Jo...
Name: sentence, Length: 350000, dtype: object

In [ ]:
data_train.sentence.iloc[2]

'This is not an easy girl meets boy, they giggle and flirt, and live happily ever story.'

In [ ]:
import warnings
warnings.filterwarnings("ignore")

reviews = []
labels = []
texts = []

for idx in range(0,len(data_train.sentence)):
    text = data_train.sentence.iloc[idx]
    text = BeautifulSoup(text, "lxml")
    text = clean_str(text.get_text())
    texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    reviews.append(sentences)

    labels.append(data_train.review_rating.iloc[idx])

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)           ## top nb words in the reviews

In [ ]:
len(tokenizer.word_index)

69266

In [ ]:
data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

for i, sentences in enumerate(reviews):
    for j, sent in enumerate(sentences):
        if j< MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k=0
            for _, word in enumerate(wordTokens):
                if k<MAX_SENT_LENGTH and tokenizer.word_index[word]<MAX_NB_WORDS:
                    data[i,j,k] = tokenizer.word_index[word]
                    k=k+1

word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

labels = to_categorical(np.asarray(labels))
labels = to_categorical(np.asarray(labels), num_classes = 5)
labels = labels.reshape([-1,5])
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Total 69266 unique tokens.
Shape of data tensor: (350000, 25, 100)
Shape of label tensor: (2100000, 5)


In [ ]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [ ]:
## download glove embeddings from https://nlp.stanford.edu/projects/glove/
GLOVE_DIR = "/content/drive/MyDrive/CIS520 Final Project/"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors.' % len(embeddings_index))

Total 400001 word vectors.


In [ ]:
# building Hierachical Attention network
embedding_matrix = np.random.random((MAX_NB_WORDS + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i < MAX_NB_WORDS:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [ ]:
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializers.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        self.supports_masking = True
        self.mask_zero = True
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        self.W = self.init((input_shape[-1],))
        #self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)

    def call(self, x, mask=None):
        eij = K.tanh(dot_product(x, self.W))

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

In [ ]:
#tf.compat.v1.disable_eager_execution()

In [ ]:
#tf.compat.v1.placeholder(
#    tf.float64
#)

In [ ]:
embedding_layer = Embedding(MAX_NB_WORDS + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,
                            trainable=False)

sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(GRU(50, return_sequences=True))(embedded_sequences)
l_dense = TimeDistributed(Dense(100))(l_lstm)
l_att = AttLayer()(l_dense)
sentEncoder = Model(sentence_input, l_att)
sentEncoder.summary()

review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype = 'int32')
#review_encoder = TimeDistributed(sentEncoder.get_layer(index = 0))(review_input)
review_encoder = TimeDistributed(sentEncoder)(review_input)
#review_encoder = tf.reshape(review_encoder, [-1,25,100])
l_lstm_sent = Bidirectional(GRU(50, return_sequences=True))(review_encoder)
l_dense_sent = TimeDistributed(Dense(100))(l_lstm_sent)
l_att_sent = AttLayer()(l_dense_sent)
preds = Dense(5, activation='softmax')(l_att_sent)
model = Model(review_input, preds)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 100)          2000100   
                                                                 
 bidirectional (Bidirectiona  (None, 100, 100)         45600     
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 100, 100)         10100     
 ibuted)                                                         
                                                                 
 att_layer (AttLayer)        (None, 100)               0         
                                                                 
Total params: 2,055,800
Trainable params: 55,700
Non-trainabl

In [ ]:
print('GPU name: ', tf.config.experimental.list_physical_devices('GPU'))


GPU name:  []


In [ ]:
def custom_f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))

        recall = TP / (Positives+K.epsilon())
        return recall


    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = TP / (Pred_Positives+K.epsilon())
        return precision

    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)

    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## Train Model

In [ ]:
from keras.metrics import AUC, Recall, Precision

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy', Precision(), Recall(), AUC(), custom_f1])

callback = [EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=1,
                              verbose=0, mode='auto')]

print("model fitting - Hierachical attention network")
print(sentEncoder.summary())
print(model.summary())

#model.fit(x_train, y_train, validation_data=(x_val, y_val), 
#          epochs=1, batch_size=100, callbacks = callback)

model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=1, batch_size=100, callbacks = callback)

model.save('/content/drive/MyDrive/CIS520 Final Project/')

model fitting - Hierachical attention network
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 100, 100)          2000100   
                                                                 
 bidirectional_2 (Bidirectio  (None, 100, 100)         45600     
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 100, 100)         10100     
 tributed)                                                       
                                                                 
 att_layer_2 (AttLayer)      (None, 100)               0         
                                                                 
Total params:

## Test Model

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/CIS520 Final Project/', custom_objects={'custom_f1':custom_f1})

In [ ]:
X_test = pd.read_pickle('/content/drive/MyDrive/CIS520 Final Project/X_test.pkl')
y_test = pd.read_pickle('/content/drive/MyDrive/CIS520 Final Project/y_test.pkl')
data_test = pd.concat([X_test, y_test], axis = 1)
data_train = data_test

import warnings
warnings.filterwarnings("ignore")

reviews = []
labels = []
texts = []

for idx in range(0,len(X_test.sentence)):
    text = X_test.sentence.iloc[idx]
    text = BeautifulSoup(text, "lxml")
    text = clean_str(text.get_text())
    texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    reviews.append(sentences)

    labels.append(X_test.review_rating.iloc[idx])

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)           ## top nb words in the reviews

data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

for i, sentences in enumerate(reviews):
    for j, sent in enumerate(sentences):
        if j< MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k=0
            for _, word in enumerate(wordTokens):
                if k<MAX_SENT_LENGTH and tokenizer.word_index[word]<MAX_NB_WORDS:
                    data[i,j,k] = tokenizer.word_index[word]
                    k=k+1

word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

labels = to_categorical(np.asarray(labels))
labels = to_categorical(np.asarray(labels), num_classes = 5)
labels = labels.reshape([-1,5])
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# building Hierachical Attention network
embedding_matrix = np.random.random((MAX_NB_WORDS + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i < MAX_NB_WORDS:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(MAX_NB_WORDS + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,
                            trainable=False)

sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_test = data
y_test = labels

#X_test = embedded_sequences

Total 29011 unique tokens.
Shape of data tensor: (50000, 25, 100)
Shape of label tensor: (300000, 5)


In [ ]:
y_pred = model.predict(X_test)

from keras.metrics import AUC, Recall, Precision, Accuracy

y_pred = y_pred.flatten()
y_test = y_test.flatten()

print('AUC: ', AUC(y_pred, y_test))
print('Accuracy: ', Accuracy(y_pred, y_test))
print('Precision: ', Precision(y_pred, y_test))
print('Recall: ', Recall(y_pred, y_test))
print('F1: ', custom_f1(y_pred, y_test))

1563/1563 [==============================] - 688s 440ms/step


ValueError: ignored

In [ ]:
from sklearn.metrics import roc_auc_score as AUC, accuracy_score as Accuracy, precision_score as Precision,recall_score as Recall

print('AUC: ', AUC(y_test, y_pred))
#print('Accuracy: ', Accuracy(y_test, y_pred, ))
#print('Precision: ', Precision(y_test, y_pred))
#print('Recall: ', Recall(y_test, y_pred))
print('F1: ', custom_f1(y_test, y_pred))

AUC:  0.9584553622
F1:  tf.Tensor(0.83334, shape=(), dtype=float32)


In [ ]:
y_test.shape

(250000,)

In [ ]:
y_pred.shape

(250000,)

In [ ]:
test_data = np.zeros((1, MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
## sample reviews

## rating = 1
#rev = 'Described as "super soft" and "Exceptionally heavy weight". But what I received looked more like a set of cheap towels coming from a discount store, and looked neither "super soft" nor "Exceptionally heavy weight". My order came from Warehouse Deals and was packed in a transparent plastic bag, so maybe a mistake was made?Anyway they are being sent back..'

## rating = 4
#rev = "I guess after looking at other reviews that we got lucky  to have great service. Nicole was our server and she first of all informed that the computers were down so orders were going in slowly. Secondly, my wife ordered a lamb burger to be cooked medium and when it came out it was almost raw. Nicole took it back immediately and five minutes later my wife had a perfect lamb burger. On top of that Nicole gave us a free desert because of the burger and the wait.  My burger was great and with service like this who wouldn't come back!"

## rating = 3
#rev = "We have at least 2 birthday parties to attend here a year. The good part is the fact that the rides are contained in one secluded area so it's easy to keep and eye on the kiddos. The bad part is the cheesy arcade located within. Don't get me wrong I don't mind arcade games, what I detest is the stupid ticket program. You know, the one where the kids collect tickets from the various arcade games then turn them in for a prize? The program here is ridiculous. You have to get and ungodly amount of tickets to get anything, and that \"anything\" is usually something that can be purchased at the dollar store. The other annoying factor is that the ticket redemption booth is always understaffed and there you are...waiting in a line of indecisive kids so that your child can pick one of the many booby prizes available. It's torture I tell you! If they got rid of this ticket BS I would frequent this place 10 times more. My kids love the go carts and the kiddie roller coaster."

## rating = 2
rev = "Will defiantly be reading more of aly's work"
revs = []
word_atten_sen_number = 2

text = BeautifulSoup(rev, "lxml")
text = clean_str(text.get_text())
revs.append(tokenize.sent_tokenize(text))

for i, sentences in enumerate(revs):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k=0
            for _, word in enumerate(wordTokens):
                if k<MAX_SENT_LENGTH and tokenizer.word_index[word]<MAX_NB_WORDS:
                    test_data[i,j,k] = tokenizer.word_index[word]
                    k=k+1

test_words = []
test_data_words = np.zeros((1, MAX_SENT_LENGTH), dtype='int32')
for i, sentences in enumerate(revs):
    for j, sent in enumerate(sentences):
        if(j!=word_atten_sen_number):
            continue
        print(sent)
        wordTokens = text_to_word_sequence(sent)
        k=0
        for _, word in enumerate(wordTokens):
            if k<MAX_SENT_LENGTH and tokenizer.word_index[word]<MAX_NB_WORDS:
                test_data_words[i,k] = tokenizer.word_index[word]
                k=k+1
                test_words.append(word)
        break

In [ ]:
model.layers[2].output

<KerasTensor: shape=(None, 25, 100) dtype=float32 (created by layer 'bidirectional_3')>

In [ ]:
sent_atten_output = K.function([model.layers[0].input], [model.layers[2].output])
output = sent_atten_output([test_data])[0]  # test mode
print(output[0].shape)

(25, 100)


In [ ]:
model.layers[-2].output

<KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'att_layer_3')>

In [ ]:
eij = np.tanh(np.multiply(model.layers[-2].get_weights(), np.transpose(output[0])))
ai = np.exp(eij)
for i in range(len(np.sum(test_data[0], axis = 1))):
    if np.sum(test_data[0], axis = 1)[i] == 0:
        ai[0][i] = 0.0
weights = ai/np.sum(ai)

print(weights[0][:len(revs[0])])
print(len(revs[0]))

ValueError: ignored

In [ ]:
list(zip(revs[0], weights[0][:len(revs[0])]))

NameError: ignored

In [ ]:
word_atten_output = K.function([sentEncoder.layers[0].input, K.learning_phase()], [sentEncoder.layers[2].output])
out = word_atten_output([test_data_words, 0])[0]  # test mode
print(out[0].shape)

In [ ]:
eij = np.tanh(np.dot(sentEncoder.layers[-1].get_weights(), np.transpose(out[0])))
ai = np.exp(eij)
count=0
for i in range(len(np.sum(test_data_words, axis = 0))):
    if np.sum(test_data_words, axis = 0)[i] == 0:
        ai[0][i] = 0.0
    else:
        count+=1
weights = ai/np.sum(ai)

print(weights[0][:count])
print(count)

In [ ]:
list(zip(test_words, weights[0][:count]))